In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import nibabel as nib
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib.pyplot as plt
from glob import glob
import torchio as tio
from pathlib import Path

import wandb
from wandb.keras import WandbCallback

wandb.init(project="myproject")

try:

    with tf.device('/device:GPU:5'):
        
# Runs the op.

            
        demo = False
        process = False
        #UR DATA DIR
        data_dir   = '/opt/localdata/CervicalSpineKaggle/'
        #DESIRED OUTPUT DIR
        out_dir    = '/opt/localdata/CervicalSpineKaggle/Process/'

        if process:

            for dataset in ['train_images']:
                dataset_dir = f'{data_dir}{dataset}'
                patients = os.listdir(dataset_dir)
                if demo:
                    ##FIRST 5 SUBS ONLY
                    patients = patients[:2]
                
                print(f'Total patients in {dataset} dataset: {len(patients)}')

                count = 0
                for patient in patients:
                    count = count + 1
                    print(f'{dataset}: {count}/{len(patients)}')
                    scan_src  = f'{dataset_dir}/{patient}/'
                    scan_dest = f'{out_dir}/{dataset}/{patient}'
                    #Path(scan_dest).mkdir(parents=True, exist_ok=True)
                    image = tio.ScalarImage(scan_src)
                    HOUNSFIELD_AIR, HOUNSFIELD_BONE = -1000, 1000
                    clamp=tio.Clamp(out_min=HOUNSFIELD_AIR, out_max=HOUNSFIELD_BONE)
                    image=clamp(image)
                    transforms = [
                        ##Standardize Orientation
                        tio.ToCanonical(),
                        ##Resample to 1mm isotropic voxels bspline
                        tio.Resample(1),           
                        ## Crop or pad to 140/140/220
                        tio.CropOrPad((160,160,224)),
                        ##Rescale from -1 to 1 at -1000 to 1000 HU
                        tio.RescaleIntensity(out_min_max = [0,1], in_min_max = [0, 800])
                    ]
                    transform = tio.Compose(transforms)
                    preprocessed = transform(image)
                    preprocessed.save(f'{scan_dest}.nii.gz')
                    
        import random

        def tio_augment(vol_base):
            
            transforms_dict = {
            tio.RandomAffine(scales=(0.90, 1.1),degrees=(-25,25)): 0.33,
            tio.RandomElasticDeformation(num_control_points=(8, 8, 8),locked_borders=2): 0.33,
            tio.RandomFlip(axes=('LR',)): 0.33,
            } 
            transform = tio.OneOf(transforms_dict)
            transformed_b = transform(vol_base)
            transformed_b = transformed_b.data
        
            return transformed_b

        ##This is the augmentation function workhorse
        def tio_augment_wlabel(subj):
            transforms_dict = {
            tio.RandomAffine(scales=(0.90, 1.1),degrees=(-45,45)): 0.5,
            tio.RandomElasticDeformation(num_control_points=(8, 8, 8),locked_borders=2): 0.5,
            } 
            transform = tio.OneOf(transforms_dict)
            subj_aug = transform(subj)
            transforms_dictflip = {
            tio.RandomFlip(axes=('LR',)): 0.25,
            tio.RandomFlip(axes=('IS',)): 0.5,
            tio.RandomFlip(axes=('AP',)): 0.25,
        } 
            transform = tio.OneOf(transforms_dictflip)
            subj_aug = transform(subj_aug)
            transform = tio.Clamp(out_min=0, out_max=1)
            subj_aug = transform(subj_aug)
            return subj_aug

        def read_nifti_file(filepath):
            scan = tio.ScalarImage(filepath)
            return scan

        def train_augmentation_wlabel(path_base, path_label):
            subj = tio.Subject(
            base=tio.ScalarImage(path_base),
            label=tio.LabelMap(path_label)
            )
            subj_aug= tio_augment_wlabel(subj)
            aug_vol_b=subj_aug.base
            aug_vol_l=subj_aug.label
            aug_vol_b=np.squeeze(aug_vol_b.data)
            aug_vol_l=np.squeeze(aug_vol_l.data)
            return aug_vol_b, aug_vol_l

        def validation_wlabel(path_base, path_label):
            subj_aug = tio.Subject(
            base=tio.ScalarImage(path_base),
            label=tio.LabelMap(path_label)
            )
            aug_vol_b=subj_aug.base
            aug_vol_l=subj_aug.label
            aug_vol_b=np.squeeze(aug_vol_b.data)
            aug_vol_l=np.squeeze(aug_vol_l.data)
            return aug_vol_b, aug_vol_l

        ##LOSS FUNCTIONS
        def dice_coe(y_true,y_pred, loss_type='jaccard', smooth=1.):
            y_true_f = tf.reshape(y_true,[-1])
            y_pred_f = tf.reshape(y_pred,[-1])
            intersection = tf.reduce_sum(y_true_f * y_pred_f)
            if loss_type == 'jaccard':
                union = tf.reduce_sum(tf.square(y_pred_f)) + tf.reduce_sum(tf.square(y_true_f))
            elif loss_type == 'sorensen':
                union = tf.reduce_sum(y_pred_f) + tf.reduce_sum(y_true_f)
            else:
                raise ValueError("Unknown `loss_type`: %s" % loss_type)
            return (2. * intersection + smooth) / (union + smooth)

        def dice_loss(y_true,y_pred, loss_type='jaccard', smooth=1.):
            y_true_f = tf.cast(tf.reshape(y_true,[-1]),tf.float32)
            y_pred_f =tf.cast(tf.reshape(y_pred,[-1]),tf.float32)
            intersection = tf.reduce_sum(y_true_f * y_pred_f)
            if loss_type == 'jaccard':
                union = tf.reduce_sum(tf.square(y_pred_f)) + tf.reduce_sum(tf.square(y_true_f))
            elif loss_type == 'sorensen':
                union = tf.reduce_sum(y_pred_f) + tf.reduce_sum(y_true_f)
            else:
                raise ValueError("Unknown `loss_type`: %s" % loss_type)
            return (1-(2. * intersection + smooth) / (union + smooth))

        def read_nifti_file(filepath):
            scan = tio.ScalarImage(filepath)
            return scan

        class DataGenerator(keras.utils.Sequence):
            'Generates data for Keras'
            def __init__(self, list_scans, batch_size, dim,
                        n_classes, shuffle):
                'Initialization'
                self.dim = dim
                self.batch_size = batch_size
                self.list_scans = list_scans
                self.n_classes = n_classes
                self.shuffle = shuffle
                self.on_epoch_end()

            def __len__(self):
                'Denotes the number of batches per epoch'
                return int(np.floor(len(self.list_scans) / self.batch_size))

            def __getitem__(self, index):
                'Generate one batch of data'
                # Generate indexes of the batch
                indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
                # Find list of IDs
                list_scans_temp = [self.list_scans[k] for k in indexes]
                #labels_temp=[self.labels[k] for k in indexes]
                
                # Generate data
                #X, y = self.__data_generation(list_scans_temp,labels_temp)
                X= self.__data_generation(list_scans_temp)
                return X
                #return X,y

            def on_epoch_end(self):
                'Updates indexes after each epoch'
                self.indexes = np.arange(len(self.list_scans))
                if self.shuffle == True:
                    np.random.shuffle(self.indexes)

            #def __data_generation(self, list_scans_temp,labels_temp):
            def __data_generation(self, list_scans_temp):
                'Generates data containing batch_size samples' # X : (n_samples, *dim)
                # Initialization
                X = np.empty((self.batch_size, *self.dim))
                #y = np.empty((self.batch_size, *self.dim))
                # Generate data
                for i, ID in enumerate(list_scans_temp):
                    # Store Image
                # Xtmp, ytmp= read_nifti_file(ID,labels_temp[i])
                    Xtmp= read_nifti_file(ID)
                    #Xtmp=tio.ToCanonical()(Xtmp)
                    #transform = tio.Compose(transforms)
                    #Xtmp = transform(Xtmp)
                    Xtmp=np.squeeze(Xtmp.data)
                    X[i]=Xtmp
                    #y[i]=ytmp

                return X
            
        seg_convert=False
        patients =glob(f"{out_dir}train_images/*.nii.gz")
        count=0

        d=[]
        if seg_convert:
            try:
                with tf.device('/device:GPU:2'):
        #params = {'batch_size': 1,
        #        'dim': (160,160,224),
        #        'n_classes': 9,
        #        'shuffle': False}
                
        #test_generator= DataGenerator(patients, **params)
                    model_3d_unet=keras.models.load_model('/home/tsteed/RSNA_CHALLENGE/3dUnet_save.h5', custom_objects={'dice_loss': dice_loss, 'dice_coe': dice_coe })             
                    for patient in patients:
                        count = count + 1
                        print(f'Segmenting   {patient}...: {count}/{len(patients)}')
                
                        #model_3d_unet=keras.models.load_model('/home/tsteed/RSNA_CHALLENGE/3dUnet_save.h5', custom_objects={'dice_loss': dice_loss, 'dice_coe': dice_coe })  
                        #patients = patients[:10]
                        x=read_nifti_file(patient)
                        y=x.data
                        y=np.array(y)
                
        
                        out2=model_3d_unet.predict(y)
                        y=x
                        y.data=out2[:,:,:,:,0]
                        shortpath=patient.split('/')[-1]
                        suid=(shortpath[:len(shortpath)-7])
                        save_location=f"/opt/localdata/CervicalSpineKaggle/pred_segs/{suid}.nii.gz"
                        d.append(save_location)
                        y.save(save_location)
                    
            except RuntimeError as e:
                    print(e)

        segs_patients =glob(f"{data_dir}pred_segs/*.nii.gz")


        class Channel_3D_DataGenerator(keras.utils.Sequence):
            'Generates data for Keras'
            def __init__(self, list_scans, list_scans2, labels, batch_size, dim,
                        n_classes, shuffle):
                'Initialization'
                self.dim = dim
                self.batch_size = batch_size
                self.labels = labels
                self.list_scans = list_scans
                self.list_scans2 = list_scans2
                self.n_classes = n_classes
                self.shuffle = shuffle
                self.on_epoch_end()

            def __len__(self):
                'Denotes the number of batches per epoch'
                return int(np.floor(len(self.list_scans) / self.batch_size))

            def __getitem__(self, index):
                'Generate one batch of data'
                # Generate indexes of the batch
                indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
                #indexes = self.indexes[index:(index+(1*self.batch_size))]
                # Find list of IDs
                list_scans_temp = [self.list_scans[k] for k in indexes]
                list_scans2_temp=[self.list_scans2[k] for k in indexes]
                labels_temp=[self.labels[k] for k in indexes]
                

                # Generate data
                [X, z], y = self.__data_generation(list_scans_temp, list_scans2_temp,labels_temp)

                return [X,z],y

            def on_epoch_end(self):
                'Updates indexes after each epoch'
                self.indexes = np.arange(len(self.list_scans))
                if self.shuffle == True:
                    np.random.shuffle(self.indexes)

            def __data_generation(self, list_scans_temp, list_scans2_temp,labels_temp):
                'Generates data containing batch_size samples' # X : (n_samples, *dim, )
                # Initialization
                X = np.empty((self.batch_size, *self.dim,1))
                y = np.empty((self.batch_size,self.n_classes), dtype=int)
                z = np.empty((self.batch_size, *self.dim,1))
                # Generate data
                for i, ID in enumerate(list_scans_temp):
                    # Store Image
                    #X[i,], z[i,] = train_augmentation_wlabel(ID, list_scans2_temp[i])
                    babx, babz = train_augmentation_wlabel(ID, list_scans2_temp[i])
                    
                    X[i,]=np.expand_dims(babx, axis=3)
                    z[i,]=np.expand_dims(babz, axis=3)
                    # Store class
                    y[i] = labels_temp[i]

                return [X,z],y
            
        # Load metadata


        train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
        train_bbox = pd.read_csv(os.path.join(data_dir,"train_bounding_boxes.csv"))
        test_df = pd.read_csv(os.path.join(data_dir,"test.csv"))
        ss = pd.read_csv(os.path.join(data_dir,"sample_submission.csv"))

        # Print dataframe shapes
        print('train shape:', train_df.shape)
        print('train bbox shape:', train_bbox.shape)
        print('test shape:', test_df.shape)
        print('ss shape:', ss.shape)
        print('')
        

        #out_dir = "/media/doktoroso/Exos18/gyrixis/projects/RSNA_Spine_processed"
        x = np.array(patients)
        z = np.array(segs_patients)

        unx,unidx=np.unique(x, return_index=True)
        indices = np.random.permutation(len(unx))
        training_idx, test_idx = indices[:round(0.9*len(indices))], indices[round(0.9*len(indices))+1:]
        training_idx = np.in1d(x, unx[training_idx])
        test_idx=np.in1d(x, unx[test_idx])
        training_idx.astype(int)
        x_train, x_val = x[training_idx], x[test_idx]
        z_train, z_val = z[training_idx], z[test_idx]

        res=[]
        for study in train_df['StudyInstanceUID']:
            res.append(os.path.join(out_dir, "train_images", study, ".nii.gz"))


        train_df['image_path']=res
        ## Index train_DF for overal fracture ==> C7 binary to create labels
        y=np.array(train_df.iloc[:,[1,2,3,4,5,6,7,8]])
        #y=np.array(train_df.iloc[:,[1]])


        y_train, y_val = y[training_idx], y[test_idx]
        print(
            "Number of samples in train and validation are %d and %d."
            % (y_train.shape[0], y_val.shape[0])
        )


        #setnumberofprediction labels
        nclasses=y_train.shape[1]

        # Generators
        ## Parameters
        params = {'batch_size': 12,
                'dim': (160,160,224),
                'n_classes': nclasses,
                'shuffle': True}

        training_generator = Channel_3D_DataGenerator(x_train, z_train, y_train, **params)
        validation_generator= Channel_3D_DataGenerator(x_val, z_val, y_val, **params)



    #kernel_initializer='he_normal'
    #from keras.regularizers import l2
    #kernel_regularizer=l2(1e-4)
    #CHANNEL_AXIS = 3
        def resnet_3d():
            #kernel_initializer='he_normal'
            from keras.regularizers import l2
            kernel_regularizer=l2(1e-4)
            width=160
            height=160
            depth=224
        #length=z_train.shape[1]
            kernel_initializer='he_normal'

            inputimg = tf.keras.Input((width, height, depth, 1), name="inputimg")
            inputseg = tf.keras.Input((width, height, depth, 1), name="inputseg")
        
            xseg = layers.Lambda(lambda x: x * 0.1)(inputseg)
            #masterin=layers.Concatenate()([inputimg, inputseg])
            masterin=layers.Multiply()([inputimg,xseg])
            ## First layer
            conv1 = layers.Conv3D(filters=32, kernel_size=(5, 5, 5),
                            strides=(2,2,2), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(inputimg)
            #print(conv1.shape)
            conv11 = layers.Conv3D(filters=32, kernel_size=(5, 5, 5),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(conv1)
        
            norm1 = layers.BatchNormalization(axis=-1)(conv11)
            relu1 = layers.Activation("relu")(norm1)
            #print(relu1.shape)
            residual1 = layers.Conv3D(filters=32, kernel_size=(3, 3, 3),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(relu1)
            #print(residual1.shape)
            resblock1 = layers.add([conv1, residual1])
        
            conv2 = layers.Conv3D(filters=64, kernel_size=(5, 5, 5),
                            strides=(2,2,2), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(resblock1)
        
            conv22 = layers.Conv3D(filters=64, kernel_size=(5, 5, 5),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(conv2)

        
            norm2 = layers.BatchNormalization(axis=-1)(conv22)
            relu2 = layers.Activation("relu")(norm2)
            #print(relu1.shape)
            residual2 = layers.Conv3D(filters=64, kernel_size=(3, 3, 3),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(relu2)
            #print(residual1.shape)
            resblock2 = layers.add([conv2, residual2])
        
        
            conv3 = layers.Conv3D(filters=64, kernel_size=(3, 3, 3),
                            strides=(2,2,2), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(resblock2)
        
            conv33 = layers.Conv3D(filters=128, kernel_size=(3, 3, 3),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(conv3)

        
            norm3 = layers.BatchNormalization(axis=-1)(conv3)
            relu3 = layers.Activation("relu")(norm3)
            #print(relu1.shape)
            residual3 = layers.Conv3D(filters=64, kernel_size=(3, 3, 3),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(relu3)
        #print(residual1.shape)
            resblock3 = layers.add([conv3, residual3])
        
            conv4 = layers.Conv3D(filters=16, kernel_size=(3, 3, 3),
                            strides=(2,2,2), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(resblock3)
        
            conv44 = layers.Conv3D(filters=16, kernel_size=(3, 3, 3),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(conv4)

        
            norm4 = layers.BatchNormalization(axis=-1)(conv44)
            relu4 = layers.Activation("relu")(norm4)
            #print(relu1.shape)
            residual4 = layers.Conv3D(filters=16, kernel_size=(3, 3, 3),
                            strides=(1,1,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(relu4)
            #print(residual1.shape)
            resblock4 = layers.add([conv4, residual4])
        
            conv5 = layers.Conv3D(filters=16, kernel_size=(3, 3, 3),
                            strides=(2,2,1), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=kernel_regularizer
                            )(resblock4)
        
        
    #     #cnn = Conv3D(32, kernel_size=(5), strides=(2), padding='same', activation='relu', kernel_initializer='he_uniform', name="conv_3d_1")(img_input)
    #     cnn = MaxPooling3D(pool_size=(2,2,2), strides=(2))(cnn)
    #     cnn = Conv3D(64, kernel_size=(3), strides=(2), padding='same', activation='relu', kernel_initializer='he_uniform', name="conv_3d_2")(img_input)
    #     cnn = MaxPooling3D(pool_size=(2,2,2), strides=(2, 2, 2))(cnn)
            flatten = layers.Flatten()(conv5)
            x = layers.Dense(512, activation='relu')(flatten)
            x = layers.Dropout(0.4)(x)
            x = layers.Dense(512, activation='relu')(x)
            outputs = layers.Dense(units=nclasses,activation="sigmoid")(x)
            model = keras.Model(
            inputs=[inputimg, inputseg],
            outputs=[outputs],
            )
            return model


        model = resnet_3d()

    ##NIFTY WAY TO CALCULATE CLASS WEIGHTS BY PROBABILITY
    #from sklearn.utils.class_weight import compute_class_weight
    #y_integers = np.argmax(y_train, axis=1)
    #class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    #d_class_weights = dict(enumerate(class_weights))

    ## UNCOMMENT FOR FUN BINARY METRICS
    #metrics = [
    #      keras.metrics.TruePositives(name='tp'),
    #      keras.metrics.FalsePositives(name='fp'),
    #      keras.metrics.TrueNegatives(name='tn'),
    #      keras.metrics.FalseNegatives(name='fn'), 
    #      keras.metrics.BinaryAccuracy(name='accuracy'),
    #     keras.metrics.Precision(name='precision'),
    #      keras.metrics.Recall(name='recall'),
    #      keras.metrics.AUC(name='auc'),
    #      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    #]
        
        
        #def comp_loss(y_true, y_pred):
            
        #    y_true=tf.squeeze(y_true)
        #    y_pred=tf.squeeze(y_pred)
        #    temp_true=tf.cast(y_true, tf.float32)
        #    added=tf.add(temp_true,y_pred)-1
        #    addedp = tf.cast(added > 0, added.dtype) *2 
        #    addedn = tf.cast(added < 0, added.dtype) *1 
        #    added=tf.add(addedp,addedn)
        #    value = float(7*tf.gather(added, 0))
        #    added=tf.tensor_scatter_nd_update(added, [0], [value])
        #    return float(1. / tf.math.reduce_sum(added, axis=None, keepdims=False, name=None))
    
        weights = [14, 2, 2, 2, 2, 2, 2, 2]

        initial_learning_rate = 0.0001
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate, decay_steps=100000, decay_rate=0.99, staircase=True
            )
        model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(
            from_logits=False),
            loss_weights=weights,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=["binary_accuracy"],
    # metrics=["binary_accuracy"],
        #metrics=metrics,
        )
       # class_weights = {0: 0.1, 1: 2}
        #Define callbacks.
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
            "3d_rsna_resnet.h5", save_best_only=True
            )
        early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=50)
    # Train the model, doing validation at the end of each epoch

        model.fit(
            training_generator,
            validation_data=validation_generator,
            callbacks=[checkpoint_cb, early_stopping_cb, WandbCallback()],
            #class_weight=class_weights,
            epochs=100
        )

        model.save("3d_rsna_resnet.h5")
except RuntimeError as e:
    print(e)

binary_accuracy,▁▆▇██▇██▇████
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▂▂▁▁▁▁▁▁▁▁▁▁
val_binary_accuracy,▁▆█▅▄▄▅▇▅▆▇▅▅
val_loss,█▃▂▂▂▃▃▂▃▃▁▃▃
GFLOPs,195.74217
best_epoch,10
best_val_loss,5.2295
binary_accuracy,0.84817
epoch,12
loss,5.39502


train shape: (2019, 9)
train bbox shape: (7217, 6)
test shape: (3, 3)
ss shape: (3, 2)

Number of samples in train and validation are 1817 and 201.
Epoch 1/100
151/151 [==============================] - ETA: 0s - loss: 10.0205 - binary_accuracy: 0.7973INFO:tensorflow:Assets written to: /home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best/assets


wandb: Adding directory to artifact (/home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best)... Done. 0.3s


151/151 [==============================] - 991s 7s/step - loss: 10.0205 - binary_accuracy: 0.7973 - val_loss: 6.0479 - val_binary_accuracy: 0.8301
Epoch 2/100
151/151 [==============================] - ETA: 0s - loss: 6.0205 - binary_accuracy: 0.8385INFO:tensorflow:Assets written to: /home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best/assets


wandb: Adding directory to artifact (/home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best)... Done. 0.3s


...

151/151 [==============================] - 984s 7s/step - loss: 5.6783 - binary_accuracy: 0.8458 - val_loss: 5.5413 - val_binary_accuracy: 0.8451
Epoch 4/100
151/151 [==============================] - ETA: 0s - loss: 5.5960 - binary_accuracy: 0.8462INFO:tensorflow:Assets written to: /home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best/assets


wandb: Adding directory to artifact (/home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best)... Done. 0.3s


151/151 [==============================] - 980s 6s/step - loss: 5.5960 - binary_accuracy: 0.8462 - val_loss: 5.3079 - val_binary_accuracy: 0.8490
Epoch 5/100
151/151 [==============================] - 996s 7s/step - loss: 5.5663 - binary_accuracy: 0.8501 - val_loss: 5.3221 - val_binary_accuracy: 0.8548
Epoch 6/100
151/151 [==============================] - 991s 7s/step - loss: 5.4639 - binary_accuracy: 0.8466 - val_loss: 5.5041 - val_binary_accuracy: 0.8405
Epoch 7/100
151/151 [==============================] - 987s 7s/step - loss: 5.4489 - binary_accuracy: 0.8496 - val_loss: 5.3081 - val_binary_accuracy: 0.8470
Epoch 8/100
151/151 [==============================] - ETA: 0s - loss: 5.4185 - binary_accuracy: 0.8500INFO:tensorflow:Assets written to: /home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best/assets


wandb: Adding directory to artifact (/home/tsteed/wandb/run-20220908_195523-38go7a8z/files/model-best)... Done. 0.3s


151/151 [==============================] - 1001s 7s/step - loss: 5.4185 - binary_accuracy: 0.8500 - val_loss: 5.3069 - val_binary_accuracy: 0.8451
Epoch 9/100
151/151 [==============================] - 991s 7s/step - loss: 5.4138 - binary_accuracy: 0.8508 - val_loss: 5.3909 - val_binary_accuracy: 0.8490
Epoch 10/100
151/151 [==============================] - 993s 7s/step - loss: 5.4080 - binary_accuracy: 0.8480 - val_loss: 5.4897 - val_binary_accuracy: 0.8431
Epoch 11/100
151/151 [==============================] - 1000s 7s/step - loss: 5.3639 - binary_accuracy: 0.8460 - val_loss: 5.6197 - val_binary_accuracy: 0.8392
Epoch 12/100
151/151 [==============================] - 974s 6s/step - loss: 5.3020 - binary_accuracy: 0.8477 - val_loss: 5.6492 - val_binary_accuracy: 0.8470
Epoch 13/100
151/151 [==============================] - 985s 7s/step - loss: 5.3480 - binary_accuracy: 0.8496 - val_loss: 5.6335 - val_binary_accuracy: 0.8451
Epoch 14/100
151/151 [==============================] - 99

In [ ]:
model = resnet_3d()

model.compile(
loss=tf.keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
     metrics=["binary_accuracy"]
)

In [ ]:
with tf.device('/device:GPU:6'):
    model = resnet_3d()

    model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["binary_accuracy"]
    )
    
    model.load_weights('/home/tsteed/wandb/latest-run/files/model-best.h5')
    a=training_generator.__getitem__(10)
    b=a[0]
    prediction = model.predict(b)

In [ ]:
a=training_generator.__getitem__(10)
b=a[0]

In [ ]:


prediction = model.predict(b)


In [ ]:
(np.round(prediction)-a[1])

In [7]:
a[1]

NameError: name 'a' is not defined

In [8]:
prediction

NameError: name 'prediction' is not defined